In [1]:
# import libaries
import numpy as np
import pandas as pd
import nltk, pprint
import matplotlib.pyplot as plt
import random
import itertools

# supress warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,multilabel_confusion_matrix
from sklearn.utils import class_weight



import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,LSTM,RepeatVector,GRU
from tensorflow.keras.layers import Dense, Input,Dropout,Bidirectional
from tensorflow.keras.layers import TimeDistributed




In [2]:
input= np.round(np.random.rand(2,4)*10,1).astype('float')
print(input)
output=np.sort(input)
print(output)

[[7.3 5.3 9.3 9.9]
 [5.6 2.5 0.4 9.2]]
[[5.3 7.3 9.3 9.9]
 [0.4 2.5 5.6 9.2]]


In [3]:
X=np.round(np.random.rand(10000,5)*10,1)
y=np.sort(X)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
X_train.shape,X_test.shape

((7000, 5), (3000, 5))

In [6]:

dig2idx={str(i):i for i in range(10)}
dig2idx['.']=10
idx2dig={v:k for k, v in dig2idx.items()}
print(dig2idx)
print(idx2dig)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '.': 10}
{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '.'}


In [7]:
# max len =4
Max_len=4
X_train.max()

10.0

In [8]:
def convert_to_categorical(arr):
  # return [[[dig2idx[dig]   for dig in str(ele) ]  for ele in sam]for sam in arr   ]
  final_arr=np.zeros((*arr.shape,4))
  for i, sam in enumerate(arr):
    for j, ele in enumerate(sam):
      row=[]
      if(len(str(ele)) !=4):
        ele=format(ele,'.2f')
      for dig in str(ele):
        row.append(dig2idx[dig])
      final_arr[i,j,:]=np.array(row)
  return final_arr
      



In [9]:
X_train=convert_to_categorical(X_train)
X_test=convert_to_categorical(X_test)
y_train=convert_to_categorical(y_train)
y_test=convert_to_categorical(y_test)

X_train=to_categorical(X_train)
X_test=to_categorical(X_test)
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [106]:
X_train.shape

(7000, 5, 4, 11)

In [107]:
# X_train[1]

## Many to Many

In [172]:
model=Sequential()
# model.add(Input([*X_train.shape][2:]))
model.add(TimeDistributed(Bidirectional(LSTM(128,return_sequences=True)),input_shape=(X_train.shape[1:])))
model.add(TimeDistributed(Bidirectional(LSTM(128,return_sequences=True))))
model.add(TimeDistributed(Dense(11, activation='softmax')))

In [173]:

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [174]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_93 (TimeDis (None, 5, 4, 256)         143360    
_________________________________________________________________
time_distributed_94 (TimeDis (None, 5, 4, 256)         394240    
_________________________________________________________________
time_distributed_95 (TimeDis (None, 5, 4, 11)          2827      
Total params: 540,427
Trainable params: 540,427
Non-trainable params: 0
_________________________________________________________________


In [175]:
model.fit(X_train,y_train,validation_split=0.2, batch_size=32,epochs=10)

Epoch 1/10
175/175 [==============================] - 8s 22ms/step - loss: 1.6707 - acc: 0.5090 - val_loss: 1.1479 - val_acc: 0.6249
Epoch 2/10
175/175 [==============================] - 1s 8ms/step - loss: 1.1340 - acc: 0.6298 - val_loss: 1.1120 - val_acc: 0.6383
Epoch 3/10
175/175 [==============================] - 1s 8ms/step - loss: 1.1127 - acc: 0.6364 - val_loss: 1.1053 - val_acc: 0.6390
Epoch 4/10
175/175 [==============================] - 1s 8ms/step - loss: 1.1082 - acc: 0.6362 - val_loss: 1.1036 - val_acc: 0.6388
Epoch 5/10
175/175 [==============================] - 2s 9ms/step - loss: 1.1060 - acc: 0.6363 - val_loss: 1.1034 - val_acc: 0.6388
Epoch 6/10
175/175 [==============================] - 1s 8ms/step - loss: 1.1074 - acc: 0.6354 - val_loss: 1.1042 - val_acc: 0.6391
Epoch 7/10
175/175 [==============================] - 1s 8ms/step - loss: 1.1060 - acc: 0.6359 - val_loss: 1.1038 - val_acc: 0.6391
Epoch 8/10
175/175 [==============================] - 1s 8ms/step - loss: 1

In [176]:
model.evaluate(X_test,y_test)

94/94 [==============================] - 0s 5ms/step - loss: 1.1023 - acc: 0.6386


[1.1022629737854004, 0.6386333107948303]

In [177]:
pred=model.predict(X_test)

In [10]:
def unpack_arr(arr):
  result=np.zeros((arr.shape[:1]))
  # for i,seq in enumerate(arr):
    # result[rand,i]=[for ] 
  for j , ele in enumerate(arr):
    # print(ele.shape)
    digit=""
    for k, dig in enumerate(ele):
      digit+=idx2dig[np.argmax(dig)]
    # print(digit)
    result[j]=float(digit)
  return result




In [179]:
X_test[0].shape
rand=np.random.randint(len(X_test))
print(rand)
print(unpack_arr(X_test[rand]))
print("-"*50)
print(unpack_arr(y_test[rand]))
print("-"*50)
print(unpack_arr(pred[rand]))

2891
[8.9 7.9 3.9 3.7 7.9]
--------------------------------------------------
[3.7 3.9 7.9 7.9 8.9]
--------------------------------------------------
[8.9 7.9 3.9 3.7 7.9]


## Seq2Seq

In [12]:
# instantiate model
model = Sequential()

# encoder with multiple inputs	
# return_sequences =False focus only after the last time step
model.add(TimeDistributed(Bidirectional(GRU(512, return_sequences=True)), input_shape=(X_train.shape[1:])))
model.add(TimeDistributed(Bidirectional(GRU(256, return_sequences=True))))
model.add(TimeDistributed(Bidirectional(GRU(64))))
# here we do not return_sequence
# if we do not use RepeatVector the output dimentions of the encoder will be 2d (None,128) but the decoder will expect (None,100,128)
# and the meaning of it will be learnt by the model during back propagatioin

# encoded sequence	
model.add(TimeDistributed(RepeatVector(4)))
		
model.add(TimeDistributed(Bidirectional(GRU(256, return_sequences=True))))
model.add(TimeDistributed(Bidirectional(GRU(64, return_sequences=True))))
# model.add(TimeDistributed(Bidirectional(LSTM(128, return_sequences=True))))

# TimeDistributed(): focus on every time step
model.add(TimeDistributed(Dense(11, activation='softmax')))

In [13]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_2 (TimeDist (None, 5, 4, 1024)        1612800   
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 4, 512)         1969152   
_________________________________________________________________
time_distributed_4 (TimeDist (None, 5, 128)            221952    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 5, 4, 128)         0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 5, 4, 512)         592896    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 5, 4, 128)         221952    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 5, 4, 11)         

In [15]:
model.fit(X_train,y_train,validation_split=0.2, batch_size=6,epochs=20)

Epoch 1/20
934/934 [==============================] - 34s 18ms/step - loss: 1.2603 - acc: 0.5578 - val_loss: 1.1428 - val_acc: 0.5951
Epoch 2/20
934/934 [==============================] - 14s 15ms/step - loss: 1.1381 - acc: 0.5963 - val_loss: 1.1342 - val_acc: 0.5996
Epoch 3/20
934/934 [==============================] - 14s 15ms/step - loss: 1.1343 - acc: 0.5984 - val_loss: 1.1324 - val_acc: 0.5998
Epoch 4/20
934/934 [==============================] - 14s 15ms/step - loss: 1.1276 - acc: 0.6045 - val_loss: 1.1275 - val_acc: 0.6147
Epoch 5/20
934/934 [==============================] - 14s 15ms/step - loss: 1.1247 - acc: 0.6124 - val_loss: 1.1241 - val_acc: 0.6164
Epoch 6/20
934/934 [==============================] - 14s 15ms/step - loss: 1.1187 - acc: 0.6211 - val_loss: 1.1170 - val_acc: 0.6279
Epoch 7/20
934/934 [==============================] - 14s 15ms/step - loss: 1.1116 - acc: 0.6318 - val_loss: 1.1102 - val_acc: 0.6355
Epoch 8/20
934/934 [==============================] - 14s 15ms

In [20]:
model.evaluate(X_test,y_test)

94/94 [==============================] - 1s 8ms/step - loss: 1.1063 - acc: 0.6361


[1.106274962425232, 0.6361333131790161]

In [21]:
pred=model.predict(X_test)

In [26]:
rand=np.random.randint(len(X_test))
print(rand)
print(unpack_arr(X_test[rand]))
print("-"*50)
print(unpack_arr(y_test[rand]))
print("-"*50)
print(unpack_arr(pred[rand]))

642
[2.1 6.5 6.6 0.3 3. ]
--------------------------------------------------
[0.3 2.1 3.  6.5 6.6]
--------------------------------------------------
[2.1 6.5 8.6 0.3 3. ]
